In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of USFWS biologist labels of imagery from Bosque del Apache and Maxwell NWR

In [1]:
#Imports
import pandas as pd
import numpy as np
import ast

import sklearn.metrics

from shapely.geometry import Polygon,Point
import matplotlib.pyplot as plt
import shapely
import cv2 as cv
import os
import gc

In [2]:
#Analysis annotations
path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/expert/20230307_expertanalysislabels_spponly.csv"
with open(path) as f:
  df = pd.read_csv(f)
  df = df[df["filename"] != "BDA_24C_20181107_1.JPG"]
  df['bbox_orig'] = df['bbox_orig'].apply(ast.literal_eval)
  #df['bbox_refined'] = df['bbox_refined'].apply(ast.literal_eval)

#Refined annotations
refinedpath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/expert/20230307_expertconsensuslabels_spponly.csv"
with open(refinedpath) as f:
  ref = pd.read_csv(f)
  ref['bbox'] = ref['bbox'].apply(ast.literal_eval)


In [3]:
spplist = list(ref["category"].unique())
spplist

['Canadian Goose',
 'Sandhill Crane',
 'Mallard',
 'Northern Pintail',
 'American Wigeon',
 'Other',
 'Teal',
 'Gadwall',
 'Northern Shoveler']

In [8]:
missing = df[df["cluster_id"] == -1]
len(missing)

1238

In [10]:
clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()
c = clusters.reset_index()
c["expected"] = c["bbox_orig"]/c["labeler"]
c["diff"] = c["cluster_id"]/c["expected"]
clusters.to_csv()
#pd.eval("c['diff'] > 1.1 or c['diff'] < 0.9").sum()

/var/folders/21/q9_tvtjj53n6mfprd_174cz00000gn/T/ipykernel_23201/3555870435.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clusters = df.groupby("filename")["bbox_orig","cluster_id","labeler"].nunique()


,bbox_orig,cluster_id,labeler
filename,,,
BDA_12C_20181127_1.JPG,841,86,10
BDA_12C_20181127_2.JPG,3817,489,8
BDA_12C_20181127_3.JPG,4867,793,6
BDA_18A4_20181106_1.JPG,1622,164,10
BDA_18A4_20181106_2.JPG,3497,356,10
BDA_18A4_20181106_3.JPG,438,63,7
BDA_18A4_20181106_4.JPG,1145,116,10
BDA_18A4_20181107_1.JPG,780,79,10
BDA_18A4_20181107_2.JPG,201,27,7


In [37]:
savepath = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/expert/"
#exportanalysis = datetime.now().strftime('%Y%m%d_zooniverseanalysislabels_seagull_nodrops.csv')
clusters.to_csv(path+"clustercheck.csv")

In [12]:
def minmax(val_list):
    min_val = min(val_list)
    max_val = max(val_list)

    return (min_val, max_val)

print(minmax(c["diff"]))
print(c["diff"].std())
print(c["diff"].var())

(0.9402985074626865, 1.0248886560125754)
0.02409051426821127
0.0005803528777068907


In [8]:
len(df["cat_orig"])

19324

In [16]:
import sklearn.metrics
#ground_truth = ["cat", "cat", "others", "cat", "others", "others","cat", "cat", "cat", "others" ]
#prediction = ["cat", "others", "cat", "cat", "others", "others","cat", "others", "others", "cat"]
pred = df["cat_orig"].astype(str)
ground = df["cat_refined"].astype(str)

#confusion_matrix = sklearn.metrics.multilabel_confusion_matrix(ground, pred)
confusion_matrices = sklearn.metrics.multilabel_confusion_matrix(ground, pred)
#for confusion_matrix in confusion_matrices:
##    disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=ground)
#    disp.plot(include_values=True, cmap="viridis", ax=None, xticks_rotation="vertical")
#    plt.show()
#accuracy = sklearn.metrics.accuracy_score(ground, pred) 
#precision = sklearn.metrics.precision_score(ground, pred)
#recall = sklearn.metrics.recall_score(ground, pred)
#F1_score = sklearn.metrics.f1_score(ground, pred) 
#print({"Accuracy":accuracy,"Precision":precision,"Sensitivity_recall": recall, "Specificity": precision,"F1_score": F1_score})

#cm_display = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)
#cm_display.plot()
#plt.show() 

In [84]:
#Cluster reliability: Group by filename, labeler-- count the number of original bounding boxes per labeler, get range/variance
df['agree'] = 'No'
df.loc[df['cat_orig'] == df["cat_refined"], 'agree'] = 'Yes'
df.groupby("labeler")["agree"].value_counts()

labeler                          agree
andrew_stetter@fws.gov           Yes       949
                                 No        196
barry_wilson@fws.gov             Yes      1669
                                 No        646
bill_johnson@fws.gov             Yes      1497
                                 No        486
dan_collins@fws.gov              Yes      1069
                                 No        326
david.butler@tpwd.texas.gov      Yes      1873
                                 No        874
jeff_sanchez@fws.gov             Yes      1460
                                 No        331
jena_moon@fws.gov                No        590
                                 Yes       486
john_vradenburg@fws.gov          Yes      1205
                                 No        221
josh_vest@fws.gov                Yes       497
                                 No         11
jude_smith@fws.gov               No       1072
                                 Yes       772
kammie_kruse@fws.gov 

In [101]:
df.groupby(["labeler","cat_refined"])["agree"].value_counts()

labeler                 cat_refined       agree
andrew_stetter@fws.gov  American Wigeon   No         3
                                          Yes        3
                        Canadian Goose    Yes      100
                        Gadwall           Yes        3
                                          No         2
                                                  ... 
steven_sesnie@fws.gov   Canadian Goose    Yes       43
                        Mallard           Yes        8
                                          No         3
                        Northern Pintail  Yes        3
                        Sandhill Crane    Yes        9
Name: agree, Length: 181, dtype: int64

In [40]:
range = df.groupby(['filename', 'labeler']).size()
r = range.reset_index()

r.rename( columns={0 :'count'}, inplace=True )
r.groupby("filename")["count"].apply(minmax)

filename
BDA_12C_20181127_1.JPG        (68, 93)
BDA_12C_20181127_2.JPG      (288, 570)
BDA_12C_20181127_3.JPG     (220, 1043)
BDA_18A4_20181106_1.JPG     (159, 165)
BDA_18A4_20181106_2.JPG     (300, 367)
BDA_18A4_20181106_3.JPG       (62, 66)
BDA_18A4_20181106_4.JPG     (113, 117)
BDA_18A4_20181107_1.JPG       (75, 83)
BDA_18A4_20181107_2.JPG       (26, 36)
BDA_18A4_20181107_3.JPG       (88, 91)
BDA_18A4_20181107_4.JPG       (83, 86)
mxw_L13_20181215_1.JPG        (38, 39)
Name: count, dtype: object

In [57]:
#Set supercategories for taxa, export to separate CSV
#spplist = ['Canadian Goose', 'Sandhill Crane', 'Mallard','Northern Pintail','American Wigeon','Other','Teal','Gadwall','Northern Shoveler']
ref['class'] = 'Duck'
ref.loc[ref['category'] == "Canadian Goose", 'class'] = 'Goose'
ref.loc[ref['category'] == "Sandhill Crane", 'class'] = 'Crane'
ref['class'].value_counts()

Duck     2183
Goose     140
Crane      52
Name: class, dtype: int64

In [23]:
#Sa'doun's confusion matrix code
def iou(ground, pred):
    x1 = ground["bbox"][0]
    y1 = ground["bbox"][1]
    w1 = ground["bbox"][2]
    h1 = ground["bbox"][3]
    
    x2 = pred["bbox_orig"][0]
    y2 = pred["bbox_orig"][1]
    w2 = pred["bbox_orig"][2]
    h2 = pred["bbox_orig"][3]

    w_intersection = min(x1 + w1, x2 + w2) - max(x1, x2)
    h_intersection = min(y1 + h1, y2 + h2) - max(y1, y2)
    if w_intersection <= 0 or h_intersection <= 0: # No overlap
        return 0
    I = w_intersection * h_intersection
    U = w1 * h1 + w2 * h2 - I # Union = Total Area - I
    return I / U

In [ ]:
def class_cm(ground,pred,iou):
  
  classes = ground['category'].unique()
  cm=np.zeros((len(classes), len(classes)+2))

  for i in range(len(pred)):
    iou_list = []
    class_list=[]
    x=ground.loc[ground['filename'] == pred.iloc[i][0]]
    for j in range(len(x)):
      iou_list.append(iou(pred.iloc[i][1],pred.iloc[i][2],pred.iloc[i][3]-pred.iloc[i][1],pred.iloc[i][4]-pred.iloc[i][2],x.iloc[j][1],x.iloc[j][2],x.iloc[j][3]-x.iloc[j][1],x.iloc[j][4]-x.iloc[j][2]))
      class_list.append(x.iloc[j][5])
    if iou_list == []:
      continue
    if max(iou_list) == 0:
      cm[int(pred.iloc[i]['class'])][-1]+=1
    else:
      cm[class_list[iou_list.index(max(iou_list))]][int(pred.iloc[i]['class'])]+=1

  for i in range(len(cm)):
    cm[i][-2]=len(ground.loc[ground['class'] == i])- np.sum(cm[i][:-1])
    
  return cm

In [31]:
#Confusion matrix for bounding boxes solution from: https://towardsdatascience.com/confusion-matrix-and-object-detection-f0cbcb634157

def evaluation(ground,pred,iou_value):
  """
  ground= array of ground-truth contours.
  preds = array of predicted contours.
  iou_value= iou treshold for TP and otherwise.
  """
  truth=np.squeeze(ground)
  preds=np.squeeze(pred)
  #we will use this function to check iou less than threshold
  def CheckLess(list1,val):
    return(all(x<=val for x in list1))

  # Using predicted output as the reference
  prob1=[]
  for i in range(len(preds)):
      f1=preds[i]
      # define a Shapely polygone for prediction i
      f1=shapely.geometry.Polygon(f1)
      # determine the radius
      f1_radius=np.sqrt((f1.area)/np.pi)
      #buffer the polygon fromt the centroid
      f1_buffered=shapely.geometry.Point(f1.centroid).buffer(f1_radius*500)
      cont=[]
      for i in range(len(truth)):
        ff=shapely.geometry.Polygon(np.squeeze(truth[i]))
        if f1_buffered.contains(ff)== True:
          iou=(ff.intersection(f1).area)/(ff.union(f1).area)  
       
          cont.append((iou))

      prob1.append(cont)

  fp=0

  for t in prob1:
    if CheckLess(t,iou_value)==True:
      fp=fp+1
    
  prob2=[]
  #loop through each groun truth instance 
  for i in range(len(truth)):
      f1=truth[i]
      f1=shapely.geometry.Polygon(f1)
      #find radius
      f1_radius=np.sqrt((f1.area)/np.pi)
      #buffer the polygon from the centroid
      f1_buffered=shapely.geometry.Point(f1.centroid).buffer(f1_radius*500)
      cont=[]
      # merge up the ground truth instance against prediction
      # to determine the IoU
      for i in range(len(preds)):
        ff=shapely.geometry.Polygon(np.squeeze(preds[i]))
        if f1_buffered.contains(ff)== True:
          #calculate IoU
          iou=(ff.intersection(f1).area)/(ff.union(f1).area)
          cont.append((iou))
      # probability of a given prediction to be contained in a
      # ground truth instance
      prob2.append(cont)
  fn=0
  tp=0
  for t in prob2:
    if np.sum(t)==0:
      fn=fn+1
    elif CheckLess(t,iou_value)==False:
      tp=tp+1
  
  #lets add this section just to print the results
  print("TP:",tp,"\t FP:",fp,"\t FN:",fn,"\t GT:",truth.shape[0])
  precision=round(tp/(tp+fp),3) 
  recall=round(tp/(tp+fn),3)
  f1= round(2*((precision*recall)/(precision+recall)),3)
  print("Precall:",precision,"\t Recall:",recall, "\t F1 score:",f1)
  
  return tp,fp,fn,precision,recall,f1

In [ ]:
#OUTPUTS

#Line graph of average identification consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Bar graph of consensus (y; % agreement) by morphology (x; duck/goose/crane). Two bars per class, one for expert, one for crowdsourced
#Line graph of average count consensus across all images (y; % agreement) by individual (x). One for expert, one for crowdsourced.  
#Line graph of consensus (y; % agreement) by density (x; consensus # of individuals per image). One line for expert, one line for crowdsourced.
